In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os

import numpy as np
import tensorflow as tf
import deepchem as dc
from deepchem.models.tensorgraph.models.graph_models import GraphConvTensorGraph
from deepchem.utils.save import load_from_disk
from deepchem.feat.graph_features import ConvMolFeaturizer, WeaveFeaturizer

input_data='./hDAT_pIC50.csv'

tasks = ['affinity']
featurizer=ConvMolFeaturizer()
#featurizer=WeaveFeaturizer()

loader = dc.data.CSVLoader(tasks=tasks, smiles_field="canonical_smiles",featurizer=featurizer)
dataset=loader.featurize(input_data)


Loading raw samples now.
shard_size: 8192
About to start loading CSV from ./hDAT_pIC50.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 3.264 s
TIMING: dataset construction took 3.797 s
Loading dataset from disk.


In [2]:
import deepchem as dc
import tempfile, shutil
from deepchem.splits.splitters import RandomSplitter

splitter=RandomSplitter()
train_data,valid_data,test_data=splitter.train_valid_test_split(dataset)



len(train_data),len(valid_data),len(test_data)

print(train_data)

TIMING: dataset construction took 0.578 s
Loading dataset from disk.
TIMING: dataset construction took 0.265 s
Loading dataset from disk.
TIMING: dataset construction took 0.261 s
Loading dataset from disk.


In [3]:
from deepchem.models.tensorgraph.models.graph_models import GraphConvTensorGraph, WeaveTensorGraph

model = GraphConvTensorGraph(len(tasks), dropout=0.5, batch_size=50, mode='regression',tensorboard=False, learning_rate=0.0001)
#model = WeaveTensorGraph(len(tasks), dropout=0.5, batch_size=50, mode='regression')


model.fit(train_data, nb_epoch=100)

from sklearn.metrics import mean_absolute_error
from deepchem.metrics import rms_score
import deepchem.hyper.grid_search

metric = dc.metrics.Metric(
    mean_absolute_error, task_averager=np.mean, mode="regression")

print("Evaluating model")
train_scores = model.evaluate(train_data, [metric])

valid_scores = model.evaluate(valid_data, [metric])


/home/nagayasu/anaconda3/envs/deepchem/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
Starting epoch 20
Starting epoch 21
Starting epoch 22
Starting epoch 23
Starting epoch 24
Starting epoch 25
Starting epoch 26
Starting epoch 27
Starting epoch 28
Starting epoch 29
Starting epoch 30
Starting epoch 31
Starting epoch 32
Starting epoch 33
Starting epoch 34
Starting epoch 35
Starting epoch 36
Starting epoch 37
Starting epoch 38
Starting epoch 39
Starting epoch 40
Starting epoch 41
Starting epoch 42
Starting epoch 43
Starting epoch 44
Starting epoch 45
Starting epoch 46
Starting epoch 47
Starting epoch 48
Starting epoch 49
Starting epoch 50
Starting epoch 51
Starting epoch 52
Starting epoch 53
Starting epoch 54
Starting epoch 55
St

In [4]:
import matplotlib
import matplotlib.pyplot as plt



In [8]:
print(train_data.y)
print(model.predict(train_data))

for i in range(10):
    print(model.predict_proba(train_data))


[[6.42829117]
 [6.32790214]
 [6.42365865]
 ...
 [6.        ]
 [6.        ]
 [6.        ]]
[[6.9511333]
 [6.7873783]
 [6.8499126]
 ...
 [6.0283217]
 [6.041586 ]
 [5.9235425]]
[[[6.9674335]]

 [[6.8575206]]

 [[7.0048122]]

 ...

 [[6.3325133]]

 [[6.3665137]]

 [[5.7774878]]]
[[[6.820111 ]]

 [[7.1223965]]

 [[7.3354883]]

 ...

 [[6.3311315]]

 [[6.131397 ]]

 [[6.1499295]]]
[[[7.2837543]]

 [[6.895457 ]]

 [[7.138221 ]]

 ...

 [[6.0067263]]

 [[6.147125 ]]

 [[6.0368137]]]
[[[7.2355185]]

 [[6.9923234]]

 [[7.303121 ]]

 ...

 [[6.4236326]]

 [[6.236051 ]]

 [[5.9858074]]]
[[[7.1506586]]

 [[7.1892385]]

 [[6.9814653]]

 ...

 [[6.534867 ]]

 [[6.3146443]]

 [[6.140617 ]]]
[[[7.10222  ]]

 [[7.0707793]]

 [[7.0819397]]

 ...

 [[6.019953 ]]

 [[6.2231   ]]

 [[6.4070916]]]
[[[7.046963 ]]

 [[6.9765615]]

 [[6.922988 ]]

 ...

 [[6.2236276]]

 [[6.062258 ]]

 [[5.963997 ]]]
[[[6.645162 ]]

 [[6.9052167]]

 [[6.952717 ]]

 ...

 [[6.20708  ]]

 [[6.4788737]]

 [[6.1343203]]]
[[[6.72535

In [20]:


metric = dc.metrics.Metric(
    dc.metrics.roc_auc_score, np.mean, mode="classification")

def reshape_y_pred(y_true, y_pred):
    """
    TensorGraph.Predict returns a list of arrays, one for each output
    We also have to remove the padding on the last batch
    Metrics taks results of shape (samples, n_task, prob_of_class)
    """
    n_samples = len(y_true)
    retval = np.stack(y_pred, axis=1)
    return retval[:n_samples]


print("Evaluating model")
train_predictions = tg.predict_on_generator(data_generator(train_dataset, predict=True))
train_predictions = reshape_y_pred(train_dataset.y, train_predictions)
train_scores = metric.compute_metric(train_dataset.y, train_predictions, train_dataset.w)
print("Training ROC-AUC Score: %f" % train_scores)

valid_predictions = tg.predict_on_generator(data_generator(valid_dataset, predict=True))
valid_predictions = reshape_y_pred(valid_dataset.y, valid_predictions)
valid_scores = metric.compute_metric(valid_dataset.y, valid_predictions, valid_dataset.w)
print("Valid ROC-AUC Score: %f" % valid_scores)

Evaluating model


NameError: name 'tg' is not defined